In [2]:
import os
from pyspark.sql import SparkSession

#spark = SparkSession.builder.master("local").appName("mllib_classifier").getOrCreate()
spark = SparkSession.builder.appName("StockPricePrediction").getOrCreate()

# Load training data
filename = "KOv3.csv"
data = spark.read.csv(filename, inferSchema=True, header = True)
#data.show(2)

In [3]:
data.describe

<bound method DataFrame.describe of DataFrame[Date: string, Open: double, High: double, Low: double, Close: double, Adj Close: double, PercentChangeClose: double, Percent100: double, Volume: int, MarketCap: bigint, EnterpriseValue: bigint, PeRatio: double, PsRatio: double, PbRatio: double, EnterprisesValueRevenueRatio: double, EnterprisesValueEBITDARatio: double, TotalRevenue: bigint, OperatingRevenue: bigint, CostOfRevenue: bigint, GrossProfit: bigint, OperatingExpense: bigint, SellingGeneralAndAdministration: bigint, OperatingIncome: bigint, OtherIncomeExpense: bigint, PretaxIncome: bigint, TaxProvision: bigint, NetIncomeCommonStockholders: bigint, NetIncome: bigint, NetIncomeIncludingNoncontrollingInterests: bigint, NetIncomeContinuousOperations: bigint, DilutedNIAvailtoComStockholders: bigint, BasicEPS: double, DilutedEPS: double, BasicAverageShares: bigint, DilutedAverageShares: bigint, TotalOperatingIncomeAsReported: bigint, TotalExpenses: bigint, NetIncomeFromContinuingAndDiscon

In [ ]:
# Use pyspark to add ROI and buy/sell columns

In [3]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("SubsetColumns").getOrCreate()

# Define a list of columns you want to include in the subset
selected_columns = ["Date", "Adj Close", "PercentChangeClose", "Percent100", "PeRatio", "PsRatio", "PbRatio","TotalRevenue","GrossProfit", 
                    "EnterprisesValueRevenueRatio", "EnterprisesValueEBITDARatio", "OperatingExpense", 
                    "OperatingIncome", "NetIncome", "BasicEPS", "TotalExpenses", 
                    "NormalizedIncome", "EBIT", "EBITDA", "TotalAssets", "CashAndCashEquivalents", 
                    "NetPPE", "GrossPPE", "TotalDebt", "RetainedEarnings"]

# Use the select function to create a new DataFrame with the subset of columns
new_df = data.select(selected_columns)

# Show the new DataFrame
new_df.show()


23/12/01 20:39:06 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
+----------+---------+------------------+----------+-------+-------+-------+------------+-----------+----------------------------+---------------------------+----------------+---------------+----------+--------+-------------+----------------+----------+----------+-----------+----------------------+-----------+-----------+-----------+----------------+
|      Date|Adj Close|PercentChangeClose|Percent100|PeRatio|PsRatio|PbRatio|TotalRevenue|GrossProfit|EnterprisesValueRevenueRatio|EnterprisesValueEBITDARatio|OperatingExpense|OperatingIncome| NetIncome|BasicEPS|TotalExpenses|NormalizedIncome|      EBIT|    EBITDA|TotalAssets|CashAndCashEquivalents|     NetPPE|   GrossPPE|  TotalDebt|RetainedEarnings|
+----------+---------+------------------+----------+-------+-------+-------+------------+-----------+----------------------------+---------------------------+----------------

In [17]:
new_df = new_df.withColumn("ROI", new_df["PercentChangeClose"]-1)
new_df.select("PercentChangeClose","ROI").show(10,truncate=False)

+------------------+----------------------+
|PercentChangeClose|ROI                   |
+------------------+----------------------+
|0.919             |-0.08099999999999996  |
|0.973             |-0.027000000000000024 |
|1.054             |0.05400000000000005   |
|1.032             |0.03200000000000003   |
|0.939             |-0.061000000000000054 |
|1.0               |0.0                   |
|1.067             |0.06699999999999995   |
|1.091             |0.09099999999999997   |
|0.996             |-0.0040000000000000036|
|1.064             |0.06400000000000006   |
+------------------+----------------------+
only showing top 10 rows



from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Create a Spark session
spark = SparkSession.builder.appName("StringToFloatConversion").getOrCreate()

# Assuming you have a DataFrame called "your_data"
# Replace "your_data" with your actual DataFrame name
data = data  # Replace this with your actual DataFrame

# Get the list of columns in the DataFrame
all_columns = data.columns

# Iterate through each column and update its type to float
for column in all_columns:
    data = data.withColumn(column, col(column).cast("float"))

# Show the updated DataFrame
data.describe


In [ ]:
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")
scalerModel = scaler.fit(train_data)
scaledData = scalerModel.transform(train_data)

scaledData.select("high_price","features","scaledFeatures").show(2, truncate=False)

In [31]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml import Pipeline
from pyspark.ml.feature import StandardScaler

#from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Create a Spark session
#spark = SparkSession.builder.appName("StockPricePrediction").getOrCreate()

# Assuming your data is in a DataFrame called "data"
# Replace "target_column" with the actual column name of your target variable
target_column = "Adj Close"
features_columns = ["PeRatio", "PsRatio", "PbRatio","TotalRevenue","GrossProfit", 
                    "EnterprisesValueRevenueRatio", "EnterprisesValueEBITDARatio", "OperatingExpense", 
                    "OperatingIncome", "NetIncome", "BasicEPS", "TotalExpenses", 
                    "NormalizedIncome", "EBIT", "EBITDA", "TotalAssets", "CashAndCashEquivalents", 
                    "NetPPE", "GrossPPE", "TotalDebt", "RetainedEarnings"]

# Assemble the features into a vector
assembler = VectorAssembler(inputCols=features_columns, outputCol="features")
data_assembled = assembler.transform(new_df)

scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")
scalerModel = scaler.fit(data_assembled)
scaledData = scalerModel.transform(data_assembled)

# Split the data into training and testing sets
(train_data, test_data) = scaledData.randomSplit([0.8, 0.2], seed=3)

# Initialize the logistic regression model
lr = GeneralizedLinearRegression(labelCol=target_column, featuresCol="scaledFeatures",
                      family="gaussian", link="identity",
                       maxIter=10,
                      regParam=0.3) 
                      #elasticNetParam=0.8)

# Create a pipeline with the assembler and the logistic regression model
#pipeline = Pipeline(stages=[lr])

# Train the model
model = lr.fit(train_data)

# Print the weights and intercept for linear regression
print("Weights: " + str(model.coefficients))
print("Intercept: " + str(model.intercept))

# Summarize the model over the training set and print out some metrics
summary = model.summary
print("Coefficient Standard Errors: " + str(summary.coefficientStandardErrors))
print("T Values: " + str(summary.tValues))
print("P Values: " + str(summary.pValues))
print("Dispersion: " + str(summary.dispersion))
print("Null Deviance: " + str(summary.nullDeviance))
print("Residual Degree Of Freedom Null: " + str(summary.residualDegreeOfFreedomNull))
print("Deviance: " + str(summary.deviance))
print("Residual Degree Of Freedom: " + str(summary.residualDegreeOfFreedom))
print("AIC: " + str(summary.aic))
print("Deviance Residuals: ")
summary.residuals().show()



# Make predictions on the test set
#predictions = model.transform(test_data)


Weights: [-1.3657523002323226,3.022907947488744,0.8298046014627691,2.159815264116857,-2.164905933739686,0.513864503418112,0.026467450651519845,-2.6470944424845335,-1.1731383459412668,-1.0379167916912122,0.13270185802545417,3.150346087909338,1.1641720697992026,1.5417004537956427,-0.14804725064384222,7.948201727660355,0.3595092088958347,-4.4186081919064915,-2.0635762874364416,2.299482385943349,8.639551338450365]
Intercept: -5.618907933996588
Coefficient Standard Errors: [0.38712650252711656, 1.159376615277703, 0.8158312513800045, 1.9412611781284872, 1.8940869071158155, 1.1560788717615322, 0.6195454298724, 1.5168093401572444, 1.281171122164314, 1.5616643933924805, 1.6387324608008342, 1.7930606960593278, 0.934605627745882, 1.5841006679091052, 1.6851046819693136, 1.8346258346305842, 0.9275385230192119, 1.407244013867867, 1.7682753598496104, 1.184247435578401, 1.655857178491178, 1.598798351274696]
T Values: [-3.5279225041862317, 2.6073563220564644, 1.0171277455468006, 1.1125835557063328, -1.

In [45]:
from pyspark.sql import functions as F

# Extract p-values or t-values for each feature
p_values = summary.pValues
t_values = summary.tValues

coefficients_double = [float(x) for x in model.coefficients]

# Create a DataFrame with feature names, coefficients, p-values, and t-values
feature_importance_df = spark.createDataFrame(list(zip(features_columns, coefficients_double, summary.coefficientStandardErrors, p_values, t_values)),
                                             ["Feature", "Coeff", "Coefficient Std Err", "P-value", "T-value"])

# Show the feature importance DataFrame
feature_importance_df.show(100)

# You can filter the DataFrame based on p-values or t-values to identify significant features
significant_features = feature_importance_df.filter(F.col("P-value") < 0.05)

# Show significant features
significant_features.show(100)

+--------------------+--------------------+-------------------+--------------------+--------------------+
|             Feature|               Coeff|Coefficient Std Err|             P-value|             T-value|
+--------------------+--------------------+-------------------+--------------------+--------------------+
|             PeRatio| -1.3657523002323226|0.38712650252711656| 6.81840163709424E-4| -3.5279225041862317|
|             PsRatio|   3.022907947488744|  1.159376615277703|0.010795585390354745|  2.6073563220564644|
|             PbRatio|  0.8298046014627691| 0.8158312513800045| 0.31201334102836387|  1.0171277455468006|
|        TotalRevenue|   2.159815264116857| 1.9412611781284872|  0.2690612438959572|  1.1125835557063328|
|         GrossProfit|  -2.164905933739686| 1.8940869071158155| 0.25629354341650834| -1.1429813096782635|
|EnterprisesValueR...|   0.513864503418112| 1.1560788717615322|  0.6578320940090812| 0.44448913994520983|
|EnterprisesValueE...|0.026467450651519845|   

<bound method DataFrame.describe of DataFrame[Feature: string, Coefficient Std Err: double, P-value: double, T-value: double]>

In [ ]:
scaledData2.select("features","scaledFeatures").show(2, truncate=False)

In [20]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
#from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Create a Spark session
#spark = SparkSession.builder.appName("StockPricePrediction").getOrCreate()

# Assuming your data is in a DataFrame called "data"
# Replace "target_column" with the actual column name of your target variable
target_column2 = "ROI"
features_columns2 = ["PeRatio", "PsRatio", "PbRatio","TotalRevenue","GrossProfit", 
                    "EnterprisesValueRevenueRatio", "EnterprisesValueEBITDARatio", "OperatingExpense", 
                    "OperatingIncome", "NetIncome", "BasicEPS", "TotalExpenses", 
                    "NormalizedIncome", "EBIT", "EBITDA", "TotalAssets", "CashAndCashEquivalents", 
                    "NetPPE", "GrossPPE", "TotalDebt", "RetainedEarnings"]

# Assemble the features into a vector
assembler2 = VectorAssembler(inputCols=features_columns2, outputCol="features")
data_assembled2 = assembler2.transform(new_df)

# Split the data into training and testing sets
(train_data2, test_data2) = data_assembled2.randomSplit([0.8, 0.2], seed=3)

scaler2 = StandardScaler(inputCol="features", outputCol="scaledFeatures")
scalerModel2 = scaler2.fit(train_data2)
scaledData2 = scalerModel2.transform(train_data2)


# Initialize the logistic regression model
lr2 = LinearRegression(labelCol=target_column2, featuresCol="scaledFeatures",
                       maxIter=10,
                      regParam=0.3, 
                      elasticNetParam=0.8)

# Create a pipeline with the assembler and the logistic regression model
#pipeline = Pipeline(stages=[lr])

# Train the model
model2 = lr2.fit(scaledData2)

# Print the weights and intercept for linear regression
print("Weights: " + str(model2.coefficients))
print("Intercept: " + str(model2.intercept))

# Make predictions on the test set
#predictions = model.transform(test_data)

# Evaluate the model using a binary classification evaluator
#evaluator = BinaryClassificationEvaluator(labelCol=target_column)
#accuracy = evaluator.evaluate(predictions)

#print(f"Accuracy: {accuracy}")

# Optionally, you can print the coefficients of the logistic regression model
#print(f"Coefficients: {model.stages[-1].coefficients}")


Weights: [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
Intercept: 0.044452830188679245


In [11]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import functions as F

# compute predictions. this will append column "prediction" to dataframe
lrPred = model.transform(test_data)
lrPred = lrPred.withColumn("prediction", F.abs(lrPred["prediction"]))
lrPred.select("Date","Adj Close","PercentChangeClose","prediction").show(100)

ev = RegressionEvaluator(predictionCol="prediction", labelCol="Adj Close")

print("METRICS")
print("Mean Squared Error:", ev.evaluate(lrPred, {ev.metricName: "mse"}))
print("R Squared:", ev.evaluate(lrPred, {ev.metricName:'r2'}))

+----------+---------+------------------+-------------------+
|      Date|Adj Close|PercentChangeClose|         prediction|
+----------+---------+------------------+-------------------+
|12/31/1991| 4.564611|             1.167|  5.229056685695257|
|12/31/1995| 9.436478|             1.052|  9.401863230456234|
|12/31/2000|15.285267|             0.963|  15.21749241489073|
|12/31/2001|11.702862|             0.917|  13.57065186403396|
|12/31/2002|11.000153|             0.874| 12.920112308922137|
|12/31/2007|18.112333|              0.96| 16.149088025794015|
|12/31/2008|13.494187|             0.978|  13.97904493704929|
|12/31/2013|27.753237|             0.962|  32.58219526036381|
|12/31/2015|33.519585|             1.021|  36.04211018190424|
|12/31/2021|57.901939|             1.091| 45.687912149534554|
| 3/31/1990| 2.179467|             1.113|0.22579803873050874|
| 3/31/1993| 4.719271|             0.933|  5.484383010348122|
| 3/31/1996|10.234785|             1.085|   11.4571685025214|
| 3/31/2

In [6]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import functions as F

# compute predictions. this will append column "prediction" to dataframe
lrPred2 = model2.transform(test_data)
lrPred2 = lrPred2.withColumn("prediction", F.abs(lrPred2["prediction"]))
lrPred2.select("Adj Close","PercentChangeClose","Percent100","prediction").show(10)

ev2 = RegressionEvaluator(predictionCol="prediction", labelCol="PercentChangeClose")

print("METRICS")
print("Mean Squared Error:", ev2.evaluate(lrPred2, {ev2.metricName: "mse"}))
print("R Squared:", ev2.evaluate(lrPred2, {ev2.metricName:'r2'}))

+---------+------------------+----------+------------------+
|Adj Close|PercentChangeClose|Percent100|        prediction|
+---------+------------------+----------+------------------+
| 1.957785|             0.956|    95.579|1.0308207547169812|
|   2.6057|             1.196|   119.557|1.0308207547169812|
| 4.564611|             1.167|    116.73|1.0308207547169812|
| 4.719271|             0.933|    93.283|1.0308207547169812|
| 4.833506|             0.973|    97.327|1.0308207547169812|
| 5.089425|             1.026|   102.629|1.0308207547169812|
| 5.205487|             1.103|   110.303|1.0308207547169812|
| 7.204547|             1.112|   111.155|1.0308207547169812|
|  8.97204|             1.099|   109.898|1.0308207547169812|
|11.702862|             0.917|    91.718|1.0308207547169812|
+---------+------------------+----------+------------------+
only showing top 10 rows

METRICS
Mean Squared Error: 0.008227474095709678
R Squared: -0.0005303524442674679


In [ ]:
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")
scalerModel = scaler.fit(train_data)
scaledData = scalerModel.transform(train_data)

scaledData.select("high_price","features","scaledFeatures").show(2, truncate=False)

In [ ]:
from pyspark.ml.classification import LogisticRegression

# instantiate the model
lr = LogisticRegression(labelCol='high_price',
                        featuresCol='scaledFeatures',
                        maxIter=10, 
                        regParam=0.3, 
                        elasticNetParam=0.8)

# Fit the model
lrModel = lr.fit(scaledData)

# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# compute predictions. this will append column "prediction" to dataframe
lrPred = lrModel.transform(scaledData)
lrPred.select('probability','prediction').show(5,truncate=False)

# set up evaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction",
                                          labelCol="high_price",
                                          metricName="areaUnderPR")

# pass to evaluator the DF with predictions, labels
aupr = evaluator.evaluate(lrPred)

print("Area under PR Curve:", aupr)